In [ ]:
from pathlib import Path
import numpy as np
from collections import OrderedDict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
xsecs = OrderedDict(
    [
        ("qqHH_CV_1_C2V_1_C3_1", "0.0017260"),
        ("qqHH_CV_1_C2V_1_C3_0", "0.0046089"),
        ("qqHH_CV_1_C2V_1_C3_2", "0.0014228"),
        ("qqHH_CV_1_C2V_0_C3_1", "0.0270800"),
        ("qqHH_CV_1_C2V_2_C3_1", "0.0142178"),
        ("qqHH_CV_0.5_C2V_1_C3_1", "0.0108237"),
        ("qqHH_CV_1.5_C2V_1_C3_1", "0.0660185"),
    ]
)

for key in xsecs:
    xsecs[key] = eval(xsecs[key])

In [ ]:
xsecs

Get coefficients for interpolation

In [ ]:
import sympy

csamples = [
    (1, 1, 1),
    (1, 1, 0),
    (1, 1, 2),
    (1, 0, 1),
    (1, 2, 1),
    (0.5, 1, 1),
    # (1.5, 1., 1.),
]

M = sympy.Matrix(
    [
        [
            CV**2 * kl**2,
            CV**4,
            C2V**2,
            CV**3 * kl,
            CV * C2V * kl,
            CV**2 * C2V,
        ]
        for i, (CV, C2V, kl) in enumerate(csamples)
    ]
)

# the vector of couplings
CV, C2V, kl = sympy.symbols("CV C2V kl")
c = sympy.Matrix(
    [
        [CV**2 * kl**2],
        [CV**4],
        [C2V**2],
        [CV**3 * kl],
        [CV * C2V * kl],
        [CV**2 * C2V],
    ]
)

# the vector of symbolic sample cross sections
s = sympy.Matrix([[sympy.Symbol("xs{}".format(i))] for i in range(len(csamples))])

# actual computation, i.e., matrix inversion and multiplications with vectors
M_inv = M.pinv()
coeffs = c.transpose() * M_inv
sigma = coeffs * s

In [ ]:
def get_xsec_interp(cv, c2v, Kl):
    sigma_val = sigma.subs({CV: cv, C2V: c2v, kl: Kl})
    xsec = sigma_val.subs(
        {
            sympy.Symbol(f"xs{j}"): xsecs[f"qqHH_CV_{cvs}_C2V_{c2vs}_C3_{Kls}"]
            for j, (cvs, c2vs, Kls) in enumerate(csamples)
        }
    )
    return np.array(xsec)[0][0]

In [ ]:
# see: https://cmsweb.cern.ch/das/request?view=list&limit=50&instance=prod%2Fglobal&input=dataset%3D%2FVBFHHto4B_CV_*%2FRun3Summer23BPixMiniAODv4*%2FMINIAODSIM
samples = [
    (1, 0, 1),
    (1, 1, 0),
    (1, 1, 1),
    (1, 1, 2),
    (1, 2, 1),
    (1.74, 1.37, 14.4),
    (-0.012, 0.030, 10.2),
    (-0.758, 1.44, -19.3),
    (-0.962, 0.959, -1.43),
    (-1.21, 1.94, -0.94),
    (-1.60, 2.72, -1.36),
    (-1.83, 3.57, -3.39),
    (-2.12, 3.87, -5.96),
]

# see: https://github.com/LPC-HH/HH4b/blob/main/src/HH4b/xsecs.py#L165
# 13.6 TeV / 13 TeV k-factor
vbfhh_sm = 1.873758517 / 1000
# k-factor from 13.6 / 13
vbfhh_sm_prev = 1.726 / 1000
vbfhh_kfactor = vbfhh_sm / vbfhh_sm_prev
for cv, c2v, Kl in samples:
    print(
        f"(CV={cv}, C2V={c2v}, kl={Kl}) -> 13.6 TeV xsec={vbfhh_kfactor*get_xsec_interp(cv, c2v, Kl)}"
    )